In [1]:
import requests
import csv
import sys
from bs4 import BeautifulSoup as bs
from requests import get
import pandas as pd
import numpy as np
import time
from lxml import html
from tqdm import tqdm_notebook
import re

from tqdm.auto import tqdm
import pandas as pd
import requests
import glob 
import os

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time 
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from requests.exceptions import Timeout

## Modelo para raspar dados recentes

In [809]:
i = 'https://www.gov.br/inep/pt-br/composicao/quem-e-quem'

In [810]:
sopa = requests.get(i)
caldeirao = bs(sopa.text, 'lxml')
caldeirao.prettify()

'<!DOCTYPE html>\n<html lang="pt-br" xml:lang="pt-br" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <!-- Google Tag Manager -->\n  <script>\n   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n  new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\n  j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n  \'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n  })(window,document,\'script\',\'dataLayer\',\'GTM-NHZSKTD\');\n  </script>\n  <!-- End Google Tag Manager -->\n  <script type="application/ld+json">\n   {\n            "@context": "https://schema.org",\n            "@type": "Organization",\n            "url": "https://www.gov.br/inep",\n            "logo": "https://www.gov.br/inep/logo.png"\n        }\n  </script>\n  <script type="application/ld+json">\n   {\n            "@context": "https://schema.org",\n            "@type": "WebSite",\n            "url": "https://www.gov.br/inep",\n  

In [939]:
#Links Curriculo das lideranças

links_curriculo=[]
for b in caldeirao.findAll(class_='calendario'):
    p = b.get('href')
    if 'composicao' in p:
        links_curr = p
        links_curriculo.append(links_curr)
len(set(links_curriculo))

curriculos_disponíveis = []
for l in links_curriculo:
    cargo = l.split('/')[6].replace('-',' ')
    nome = l.split('/')[7].replace('-',' ')
    nome2 = l.split('/')[8].replace('-',' ')
    if '@@download' in nome2:
        nome2 = nome
    if nome == nome2:
        nome = np.nan
    curriculos_disponíveis.append([cargo, nome, nome2, l])

#salvar lista de currículos
salva = pd.DataFrame(curriculos_disponíveis, columns=['Cargo', 'Nome', 'nome2','Link_curriculo'])

In [953]:
#Baixa os currículos

def dowload_pdf_cd(url, loc=None):
    if loc is None:
            loc = os.path.basename(url.split('/')[-2].replace('-','_') +'.pdf') # pega o Nome do integrante do INEP e salva com esse nome.
            res = requests.get(url, stream=True)
            if res.status_code == requests.codes.OK:
                with open(loc, 'wb') as data:
                    for parte in res.iter_content(chunk_size=256):
                        data.write(parte)
                    else:
                        res.raise_for_status()


if __name__ == "__main__":
    
    for j in tqdm(salva['Link_curriculo']): # executa a função anterior
        dowload_pdf_cd(j)

In [56]:
e = r'C:\Users\"escreva aqui o seu diretório"'

In [57]:
# lista os endereços com os nomes dos arquivos
# junte o diretório com o nome dos arquivos, para realizar leitura.

endereco = []
for i in os.listdir('./Curriculos_Inep/'): # você poderá incluir o diretório, caso seja necessário.
    local = e + "\\" + i
    endereco.append(local)
endereco[1]

'C:\\Users\\Andre G\\Documents\\Pesquisa\\Scripts Webscraping\\Modelo Raspa (Ministérios Bozo)\\INEP\\Curriculos_Inep\\alexandre_avelino_pereira.pdf'

In [66]:
### Teste

import os
import PyPDF2
import re

tudo = []
texto = []
er = [] # lista de arquivos que não foram analisados, por algum erro.

# algorítimo que acessa os pdfs, a partir da lista de endereços gerados anteriomente.
for arquivo in tqdm(endereco):
    try:
        nome = arquivo.split('\\')[-1].replace('.pdf','') # copia o nome referente e retira o final do endereço
        object = PyPDF2.PdfFileReader(arquivo, 'rb')

        # obter o número das páginas
        NumPages = object.getNumPages()

        # termos da busca nos pdf´s
        String = "Doutor*|Mestr*"

        # extrai o texto e realiza a busca
        for i in range(0, NumPages):
            PageObj = object.getPage(i)
            Text = PageObj.extractText()
            tex = " ".join(Text.strip('\n').split())
            texto.append([tex, arquivo])
            ResSearch = re.search(String, Text, flags=re.IGNORECASE)
            if ResSearch == None:
                continue
            else:
                print(["Página: " + str(i) + " | CV => " + nome, ResSearch])
                tot = ["Página: " + str(i) + " | CV => " + nome, ResSearch]
                tudo.append(tot)

                             
    except Exception:
        error = nome
        er.append(error)
        pass
print(len(tudo)/60*100)
        


['Página: 0 | CV => alexandre_avelino_pereira', <re.Match object; span=(1355, 1360), match='Mestr'>]
['Página: 0 | CV => alexandre_bastos_peixoto', <re.Match object; span=(1320, 1325), match='Mestr'>]
['Página: 0 | CV => aline_mara_fernandes_muler', <re.Match object; span=(590, 596), match='Doutor'>]
['Página: 0 | CV => aline_perfeito_de_sousa_ramos', <re.Match object; span=(465, 471), match='Doutor'>]
['Página: 0 | CV => ana_elizabeth_maia_de_albuquerque', <re.Match object; span=(1477, 1483), match='Doutor'>]
['Página: 0 | CV => anderson_soares_furtado_oliveira', <re.Match object; span=(1333, 1338), match='Mestr'>]
['Página: 0 | CV => andreia_santos_goncalves', <re.Match object; span=(1261, 1267), match='Doutor'>]
['Página: 0 | CV => anna_priscilla_di_vasconcelos', <re.Match object; span=(1401, 1406), match='mestr'>]
['Página: 0 | CV => augusto_marques_de_castro_oliveira', <re.Match object; span=(1307, 1313), match='Doutor'>]
['Página: 0 | CV => carlos_eduardo_moreno_sampaio', <re.Mat